## Preprocessing

In [1]:
# Install kerastuner
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.1 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from kerastuner.tuners import RandomSearch
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

<ipython-input-2-c47392cab397>:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Replace APPLICATION_TYPE with 'Other' for rare types
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_type_counts[application_type_counts < 500].index.tolist()
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other")

# Replace CLASSIFICATION with 'Other' for rare types
classification_type_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_type_counts[classification_type_counts < 1800].index.tolist()
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, "Other")

# Convert categorical data to numeric
df = pd.get_dummies(application_df)

In [5]:
# Split data into features and target
target = df['IS_SUCCESSFUL']
features = df.drop(['IS_SUCCESSFUL'], axis=1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Building and Optimizing Model

In [10]:
# Function to build the model with optimization methods
def build_model(hp):
    nn = tf.keras.models.Sequential()

    # Input layer with Batch Normalization
    nn.add(Dense(
        units=hp.Int('units_input', min_value=32, max_value=128, step=16),
        activation='relu',
        input_dim=X_train_scaled.shape[1]
    ))
    nn.add(BatchNormalization())

    # Additional hidden layers with Dropout and Batch Normalization
    for i in range(hp.Int('num_layers', 1, 4)):
        nn.add(Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=16),
            activation='relu'
        ))
        nn.add(BatchNormalization())
        nn.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))

    # Output layer
    nn.add(Dense(units=1, activation='sigmoid'))

    nn.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return nn

# Hyperparameter tuning
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='my_dir',
    project_name='charity_funding_optimization'
)

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters
nn = build_model(best_hps)

# Train the model with early stopping
history = nn.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

Reloading Tuner from my_dir/charity_funding_optimization/tuner0.json
Epoch 1/100
804/804 [==============================] - 6s 4ms/step - loss: 0.5906 - accuracy: 0.7107 - val_loss: 0.5623 - val_accuracy: 0.7262
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5699 - accuracy: 0.7226 - val_loss: 0.5659 - val_accuracy: 0.7277
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5676 - accuracy: 0.7250 - val_loss: 0.5594 - val_accuracy: 0.7301
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5656 - accuracy: 0.7250 - val_loss: 0.5561 - val_accuracy: 0.7299
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5637 - accuracy: 0.7264 - val_loss: 0.5631 - val_accuracy: 0.7279
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5630 - accuracy: 0.7256 - val_loss: 0.5585 - val_accuracy: 0.7250
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss

In [12]:
# Evaluate the model
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {nn_loss}, Accuracy: {nn_accuracy}")

268/268 - 0s - loss: 0.5564 - accuracy: 0.7269 - 356ms/epoch - 1ms/step
Loss: 0.5563669800758362, Accuracy: 0.7268804907798767


In [13]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
